# Extracció d'entitats anomenades

L'objectiu d'aquesta pràctica és fer un reconeixedor d'entitats anomenades amb conditional random fields. També experimentar amb diferents "feature functions" i amb diferents tipus de codificacions. L'experimentació s'estructura de la següent forma:

- Primer provem diferents tipus de funcions per decidir quines utilitzar
- En segon lloc, provem diferents contexts en els quals aplicar aquestes funcions
- Després posem a prova ambdós models a les diferents codificacions per escollir-ne una per cada idioma
- Un cop els models estiguin fets, els provem a la partició de test

## 1 - Experimentació de "feature functions"

### Import de les dades i preprocessament

En primer lloc, importem les dades d'entrenament, validació i test de cada un dels dos idiomes. Aquestes dades venen en la codificació "BIO". Més endavant aquests tags es tractaran per provar altres codificacions.

In [1]:
import nltk
nltk.download('conll2002')
from nltk.corpus import conll2002
import warnings
warnings.filterwarnings('ignore')

# Importar datasets en español y holandés
train_es = conll2002.iob_sents('esp.train') # Train
dev_es = conll2002.iob_sents('esp.testa') # Dev
test_es = conll2002.iob_sents('esp.testb') # Test

train_ned = conll2002.iob_sents('ned.train') # Train
dev_ned = conll2002.iob_sents('ned.testa') # Dev
test_ned = conll2002.iob_sents('ned.testb') # Test

data = {'spanish': (train_es, dev_es, test_es),
        'dutch': (train_ned, dev_ned, test_ned)}

[nltk_data] Downloading package conll2002 to
[nltk_data]     C:\Users\11ser\AppData\Roaming\nltk_data...
[nltk_data]   Package conll2002 is already up-to-date!


El preprocessing el dissenyem per experimentar amb diferents combinacions de "feature functions", i aplicar-les a diferents contextos dels tokens.

Per agilitzar-ho i fer-ho de manera eficient, calcularem les "feature functions" de cada token durant el preprocessament. Això té dos avantatges principals:

1. En el cas de voler informació del context d'un token, no cal recalcular les funcions, només cal accedir a un índex diferent.
2. Per fer "grid search" estalviem temps de computació, ja que les funcions es calculen un sol cop.

Utilitzarem les 5 funcions bàsiques i afegirem 5 funcions noves:

1. Prefixos fins a longitud 3
2. El Pos-Tag del token
3. La longitud del token
4. Si el token és una stop-word
5. Si el token es troba en llistes externes de noms, organitzacions o localitzacions

In [2]:
def gazetters():
    """
    Define un conjunt de gazetteers per reconeixement d'entitats.
    Aquestes gazetteers inclouen localitzacions comunes, noms de persona, i organitzacions.
    """

    # Llista de localitzacions, noms de persona, i organitzacions
    locations_list = {
        # Ciutats principals (España i Països Baixos)
        "madrid", "barcelona", "valencia", "sevilla", "zaragoza", "bilbao", "málaga", "murcia", "palma",
        "amsterdam", "rotterdam", "utrecht", "eindhoven", "groningen", "nijmegen",

        # Regions i províncies
        "andalucía", "cataluña", "galicia", "castilla", "canarias", "baleares",
        "noord-holland", "zuid-holland", "gelderland", "limburg",

        # Països (hispanoparlants + rellevants propers)
        "españa", "méxico", "argentina", "colombia", "perú", "chile", "ecuador",
        "nederland", "belgië", "duitsland", "francia", "italia", "portugal", "reino unido", "estados unidos",

        # Geografia física
        "mediterráneo", "atlántico", "pirineos", "cantábrico", "guadalquivir", "ebro",
        "noordzee", "rijn", "maas", "veluwe"
    }

    person_names_list = {
        # Noms masculins comuns
        "juan", "josé", "david", "javier", "miguel", "pedro", "sergio", "pablo", "antonio",
        "jan", "peter", "willem", "tim", "thomas", "jeroen", "lucas", "kees",

        # Noms femenins comuns
        "maría", "ana", "isabel", "laura", "marta", "lucía", "paula", "cristina",
        "maria", "johanna", "emma", "sophie", "lisa", "lotte", "iris", "eva",

        # Cognoms comuns
        "garcía", "gonzález", "rodríguez", "fernández", "lópez", "sánchez", "martínez",
        "de jong", "jansen", "de vries", "van den berg", "bakker", "visser", "meijer"
    }

    organizations_list = {
        # Empreses destacades
        "telefónica", "bbva", "iberdrola", "mercadona", "inditex", "seat",
        "philips", "shell", "unilever", "heineken", "ing", "asml",

        # Institucions governamentals
        "gobierno", "ministerio", "ayuntamiento", "generalitat", "policía nacional", "guardia civil",
        "regering", "ministerie", "gemeente", "belastingdienst", "politie", "koninklijke marechaussee",

        # Educació i cultura
        "universidad", "museo del prado", "csic", "colegio", "hospital", "asociación", "fundación",
        "universiteit", "hogeschool", "tno", "knaw", "rijksmuseum", "omroep", "kvk"
    }

    return locations_list, person_names_list, organizations_list

In [3]:
import re
import unicodedata
import nltk
from nltk.stem import WordNetLemmatizer
from typing import List, Tuple, Optional, Callable

nltk.download('stopwords')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

def lemma(word):
    return lemmatizer.lemmatize(word)

def pos_tag(word):
    tagged = nltk.pos_tag([word])
    return tagged[0][1]

class FuncPreprocessing:
    def __init__(self, language='spanish'):
        self._pattern = re.compile(r"\d")
        if language == 'spanish':
            self.stop_words = set(nltk.corpus.stopwords.words('spanish'))
        elif language == 'dutch':
            self.stop_words = set(nltk.corpus.stopwords.words('dutch'))
        else:
            self.stop_words = set(nltk.corpus.stopwords.words('english'))
            
        # Use gazetteers
        self.locations_list, self.person_names_list, self.organizations_list = gazetters()

    def __call__(self, corpus: List[List[tuple]]) -> List[List[tuple]]:
        """
        Extract features from a corpus

        :param corpus: list of list of tuples (word, tag)
        :type corpus: list(list(tuple(str, str)))
        :return: list of features
        :rtype: list(list(list(str)))
        """
        return [self.find_sent_features(sentence) for sentence in corpus]
        
    def find_sent_features(self, sentence):
        """
        Extract features from a sentence

        :param sentence: list of tuples (word, tag)
        :type sentence: list(tuple(str, str))
        :return: list of features
        :rtype: list(list(str))
        """
        new_sentence = []
        for i in range(len(sentence)):
            new_sentence.append(((sentence[i][0], self.get_token_features(sentence, i)), sentence[i][1]))
        return new_sentence

    def get_token_features(self, tokens, idx):
        """
        Extract basic features and extra feature about this word
        """
        token = tokens[idx][0]

        feature_list = [[], [], []]

        if not token:
            return feature_list

        # Capitalization
        if token[0].isupper():
            feature_list[0].append("CAPITALIZATION")

        # Number
        if re.search(self._pattern, token) is not None:
            feature_list[0].append("HAS_NUM")

        # Punctuation
        punc_cat = {"Pc", "Pd", "Ps", "Pe", "Pi", "Pf", "Po"}
        if all(unicodedata.category(x) in punc_cat for x in token):
            feature_list[0].append("PUNCTUATION")

        # Suffix up to length 3
        if len(token) > 1:
            feature_list[0].append("SUF_" + token[-1:])
        if len(token) > 2:
            feature_list[0].append("SUF_" + token[-2:])
        if len(token) > 3:
            feature_list[0].append("SUF_" + token[-3:])

        # Word 
        feature_list[0].append("WORD_" + token)

        # Prefix up to length 3
        prefixes = []
        if len(token) > 1:
            prefixes.append("PREF_" + token[:1])
        if len(token) > 2:
            prefixes.append("PREF_" + token[:2])
        if len(token) > 3:
            prefixes.append("PREF_" + token[:3])

        feature_list[1].append(prefixes)

        # POS
        feature_list[1].append("POS_" + pos_tag(token))

        # Word length
        feature_list[1].append("WORD_LENGTH_" + str(len(token)))

        # Stop word
        if token.lower() in self.stop_words:
            feature_list[1].append("STOP_WORD")
        else:
            feature_list[1].append(None)

        # If capitalized
        if token[0].isupper():
            # Names
            if token.lower() in self.person_names_list:
                feature_list[2].append("NAME")

            # Locations
            if token.lower() in self.locations_list:
                feature_list[2].append("LOCATION")
                
            # Organizations
            if token.lower() in self.organizations_list:
                feature_list[2].append("ORGANIZATION")

        return feature_list

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\11ser\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\11ser\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
def prep_train(corpus: List[List[tuple]], language = 'spanish') -> List[List[tuple]]:
    '''
    Preprocess the corpus for training

    :param corpus: list of list of tuples (word, pos, tag)
    :type corpus: list(list(tuple(str, str, str)))
    :return: list of features
    :rtype: list(list(tuple(tuple(str, list(list(str))), str)))
    '''
    new_corpus = []
    for sentence in corpus:
        new_sentence = []
        for word_tuple in sentence:
            new_word_tuple = ((word_tuple[0], word_tuple[1]), word_tuple[2])
            new_sentence.append(new_word_tuple)
        new_corpus.append(new_sentence)
    new_corpus = FuncPreprocessing(language)(new_corpus)
    return new_corpus

def sep_token_tag(corpus: List[List[tuple]]) -> tuple[List[List[tuple]], List[List[str]]]:
    '''
    Separate the tokens and tags from the corpus
    
    :param corpus: list of list of tuples (word, tag)
    :type corpus: list(list(tuple(tuple(str, list(list(str))), str)))
    :return: tokens and tags
    :rtype: list(list(tuple)), list(list(str))
    '''
    tokens = [[word[0] for word in sentence] for sentence in corpus]
    tags = [[word[1] for word in sentence] for sentence in corpus]
    return tokens, tags

def prep_test(corpus: List[List[tuple]], language='spanish') ->tuple[List[List[tuple]], List[List[str]]]:
    '''
    Preprocess the corpus for testing
    '''
    corpus_prep = prep_train(corpus, language)
    tokens, tags = sep_token_tag(corpus_prep)
    return tokens, tags

In [5]:
# Preprocesar los datos
train_es_prep = prep_train(train_es, 'spanish')
dev_es_tokens, dev_es_tags = prep_test(dev_es, 'spanish')
test_es_tokens, test_es_tags = prep_test(test_es, 'spanish')

train_ned_prep = prep_train(train_ned, 'dutch')
dev_ned_tokens, dev_ned_tags = prep_test(dev_ned, 'dutch')
test_ned_tokens, test_ned_tags = prep_test(test_ned, 'dutch')

# Ver cómo son los datos después del preprocesamiento
print("Ejemplo de oración después del preprocesamiento:")
print(train_es_prep[0][0])

TypeError: expected string or bytes-like object, got 'tuple'

### Definició de GetFeatures

Per aprofitar el format de les dades preprocessades, creem una classe que guarda:
- Un vector "features_vector" que indica quines de les 4 primeres funcions extra s'utilitzen
- Un booleà "lists" que indica si s'utilitzen o no les llistes externes
- Un "word_vector" que indica quin context de la paraula s'utilitza (fins a dues paraules endavant i enrere)

In [6]:
class GetFeatures:
            
    def __init__(self, features_vector = [0, 0, 0, 0], lists = False, word_vector = [0, 0, 1, 0, 0]):
        self.features_vector = features_vector
        self.lists = lists
        self.word_vector = word_vector

    def __call__(self, tokens: List[tuple], idx: int) -> List[str]:
        '''
        Extract features from a token
        
        :param tokens: list of tuples (word, features)
        :type tokens: list(tuple(str, list(list(str))))
        :param idx: index of the token
        :type idx: int
        :return: list of features
        :rtype: list(str)
        '''
        feature_list = []

        if not tokens[idx][0]:
            return feature_list

        # Current word features
        if self.word_vector[2] == 1:
            features = tokens[idx][1]

            # Basic features
            for feat in features[0]:
                feature_list.append(feat)

            # Extra features
            if self.features_vector[0] == 1:  # Prefixes
                for pre in features[1][0]:
                    feature_list.append(pre)
            if self.features_vector[1] == 1:  # POS tag
                feature_list.append(features[1][1])
            if self.features_vector[2] == 1:  # Length
                feature_list.append(features[1][2])
            if self.features_vector[3] == 1:  # Stop word
                if features[1][3]:
                    feature_list.append(features[1][3])

            # External lists
            if self.lists:
                for feat in features[2]:
                    feature_list.append(feat)
            
        # 2 words before features
        if idx > 1 and self.word_vector[0] == 1:
            features = tokens[idx - 2][1]

            for feat in features[0]:
                feature_list.append("2PREV" + feat)

            if self.features_vector[0] == 1:
                for pre in features[1][0]:
                    feature_list.append("2PREV" + pre)
            if self.features_vector[1] == 1:
                feature_list.append("2PREV" + features[1][1])
            if self.features_vector[2] == 1:
                feature_list.append("2PREV" + features[1][2])
            if self.features_vector[3] == 1:
                if features[1][3]:
                    feature_list.append("2PREV" + features[1][3])

            if self.lists:
                for feat in features[2]:
                    feature_list.append("2PREV" + feat)

        # Previous word features
        if idx > 0 and self.word_vector[1] == 1:
            features = tokens[idx - 1][1]

            for feat in features[0]:
                feature_list.append("PREV" + feat)

            if self.features_vector[0] == 1:
                for pre in features[1][0]:
                    feature_list.append("PREV" + pre)
            if self.features_vector[1] == 1:
                feature_list.append("PREV" + features[1][1])
            if self.features_vector[2] == 1:
                feature_list.append("PREV" + features[1][2])
            if self.features_vector[3] == 1:
                if features[1][3]:
                    feature_list.append("PREV" + features[1][3])

            if self.lists:
                for feat in features[2]:
                    feature_list.append("PREV" + feat)

        # Next word features
        if idx < len(tokens) - 1 and self.word_vector[3] == 1:
            features = tokens[idx + 1][1]

            for feat in features[0]:
                feature_list.append("NEXT" + feat)

            if self.features_vector[0] == 1:
                for pre in features[1][0]:
                    feature_list.append("NEXT" + pre)
            if self.features_vector[1] == 1:
                feature_list.append("NEXT" + features[1][1])
            if self.features_vector[2] == 1:
                feature_list.append("NEXT" + features[1][2])
            if self.features_vector[3] == 1:
                if features[1][3]:
                    feature_list.append("NEXT" + features[1][3])

            if self.lists:
                for feat in features[2]:
                    feature_list.append("NEXT" + feat)

        # 2 words after features
        if idx < len(tokens) - 2 and self.word_vector[4] == 1:
            features = tokens[idx + 2][1]

            for feat in features[0]:
                feature_list.append("2NEXT" + feat)

            if self.features_vector[0] == 1:
                for pre in features[1][0]:
                    feature_list.append("2NEXT" + pre)
            if self.features_vector[1] == 1:
                feature_list.append("2NEXT" + features[1][1])
            if self.features_vector[2] == 1:
                feature_list.append("2NEXT" + features[1][2])
            if self.features_vector[3] == 1:
                if features[1][3]:
                    feature_list.append("2NEXT" + features[1][3])

            if self.lists:
                for feat in features[2]:
                    feature_list.append("2NEXT" + feat)

        return feature_list

### Funció d'avaluació d'un model

Abans d'experimentar amb les features, necessitem establir un criteri d'avaluació pels models. Utilitzarem dos enfocaments:

1. La "accuracy" balancejada: indica quantes etiquetes predites són correctes
2. Anàlisi d'entitats: quin percentatge d'entitats es detecten perfectament, quantes entitats "s'inventa" el model

Prioritzarem detectar entitats perfectament, que és la tasca principal del model.

In [7]:
from sklearn.metrics import balanced_accuracy_score
import re

def sent_tags_to_IO(sent_tags):
    '''
    Convert sentence tags to IO format
    
    :param sent_tags: list of tags
    :type sent_tags: list(str)
    :return: list of IO tags
    :rtype: list(str)
    '''
    io_sent_tags = []
    for sent in sent_tags:
        tags = [re.sub(r'\b[BES]-', 'I-', tag) for tag in sent]
        io_sent_tags.append(tags)
    return io_sent_tags

def entity_finder(sent_tags):
    '''
    Find entities in a sentence

    :param sent_tags: list of tags
    :type sent_tags: list(str)
    :return: list of entities
    :rtype: list(tuple(str, tuple(int, int)))
    '''
    sent_tags_io = sent_tags_to_IO(sent_tags)

    entities = []
    for sent in sent_tags_io:
        sent_entities = []
        last = sent[0]
        ini = None
        end = None
        type_ent = None
        if last[0] == 'I':
            ini = 0
            end = 0
            type_ent = last[2:]
        for i in range(1, len(sent)):
            tag = sent[i]
            if tag[0] == 'I':
                if last == 'O':
                    ini = i
                    end = i
                    type_ent = tag[2:]
                if last[0] == 'I':
                    end = i
                last = tag
            else:
                if last[0] == 'I':
                    sent_entities.append((type_ent, (ini, end)))
                    ini = None
                    end = None
                    type_ent = None
                last = tag
        if ini:
            sent_entities.append((type_ent, (ini, end)))
        entities.append(sent_entities)

    return entities

def evaluate_tagger_performance(sent_real, sent_pred, features=None, errors=False):
    '''
    Evaluate the performance of a tagger

    :param sent_real: list of real tags
    :type sent_real: list(list(str))
    :param sent_pred: list of predicted tags
    :type sent_pred: list(list(str))
    :param features: identifier for the features used
    :param errors: return errors
    :type errors: bool
    '''

    info = {'Codification': features, 'Balanced accuracy': 0, 'Total entities': 0, 'Entities correct': 0, 
            'LOC correct': 0, 'MISC correct': 0, 'ORG correct': 0, 'PER correct': 0, 'Entities invented': 0}
    
    sent_io_real = sent_tags_to_IO(sent_real)
    sent_io_pred = sent_tags_to_IO(sent_pred)

    def join_sent_tags(sent_tags):
        return [tag[0] for sent in sent_tags for tag in sent]

    # Calculate balanced accuracy
    info['Balanced accuracy'] = balanced_accuracy_score(join_sent_tags(sent_io_real), join_sent_tags(sent_io_pred))

    real_entities = entity_finder(sent_io_real)
    pred_entities = entity_finder(sent_io_pred)

    total_ent = 0
    total_loc = 0
    total_misc = 0
    total_org = 0
    total_per = 0

    for sent in real_entities:
        for entity in sent:
            total_ent += 1
            if entity[0] == 'LOC':
                total_loc += 1
            if entity[0] == 'MISC':
                total_misc += 1
            if entity[0] =='ORG':
                total_org += 1
            if entity[0] == 'PER':
                total_per += 1

    info['Total entities'] = total_ent

    good_ent = 0
    good_loc = 0
    good_misc = 0
    good_org = 0
    good_per = 0
    invented_ent = 0

    for i in range(0, len(real_entities)):
        for entity in pred_entities[i]:
            if entity in real_entities[i]:
                good_ent += 1
                if entity[0] == 'LOC':
                    good_loc += 1
                if entity[0] == 'MISC':
                    good_misc += 1
                if entity[0] =='ORG':
                    good_org += 1
                if entity[0] == 'PER':
                    good_per += 1
            else:
                invented_ent += 1

    info['Entities correct'] = good_ent/total_ent if total_ent > 0 else 0
    info['LOC correct'] = good_loc/total_loc if total_loc > 0 else 0
    info['MISC correct'] = good_misc/total_misc if total_misc > 0 else 0
    info['ORG correct'] = good_org/total_org if total_org > 0 else 0
    info['PER correct'] = good_per/total_per if total_per > 0 else 0
    info['Entities invented'] = invented_ent

    if errors:
        errors_list = []
        invented = []
        for i in range(0, len(real_entities)):
            for entity in pred_entities[i]:
                if entity not in real_entities[i]:
                    errors_list.append((i, entity))
        for i in range(0, len(pred_entities)):
            for entity in real_entities[i]:
                if entity not in pred_entities[i]:
                    invented.append((i, entity))
        return info, errors_list, invented
                
    return info

### Grid search per definir les quatre primeres funcions extres

Ara farem un "grid search" per seleccionar quines de les quatre primeres funcions extres utilitzar:

1. Prefixos
2. Pos-Tag
3. Longitud
4. Stop-Word

Provarem totes les combinacions possibles i crearem una taula per cada idioma amb els resultats.

In [8]:
import pandas as pd
import itertools
import nltk

def features_grid_search(train, val_tokens, val_tags):
    '''
    Perform a grid search over the features
    '''
    feature_combinations = list(itertools.product([0, 1], repeat=4))

    results = pd.DataFrame(columns=['Features', 'Balanced accuracy', 'Total entities', 'Entities correct', 
                                    'LOC correct', 'MISC correct', 'ORG correct', 'PER correct', 'Entities invented'])

    # Para cada combinación de características
    for features in feature_combinations:
        model = nltk.tag.CRFTagger(feature_func=GetFeatures(features_vector=features))
        model.train(train, 'crfTagger.mdl')
        prediction = model.tag_sents(val_tokens)

        _, prediction_tags = sep_token_tag(prediction)

        feat_results = evaluate_tagger_performance(val_tags, prediction_tags, features)

        results = pd.concat([results, pd.DataFrame([feat_results])], ignore_index=True)

    return results

### Anàlisi de "feature functions" en el model de l'idioma espanyol

In [9]:
# Esta operación puede tardar bastante tiempo
# results_es = features_grid_search(train_es_prep, dev_es_tokens, dev_es_tags)

# Para ahorrar tiempo, probamos solo algunas configuraciones relevantes
test_features = [[0,0,0,0], [1,0,0,0], [0,1,0,0], [0,0,1,0], [0,0,0,1], [1,1,0,0], [1,1,1,0], [1,1,1,1]]
results_es = pd.DataFrame(columns=['Features', 'Balanced accuracy', 'Total entities', 'Entities correct', 
                                  'LOC correct', 'MISC correct', 'ORG correct', 'PER correct', 'Entities invented'])

for features in test_features:
    model = nltk.tag.CRFTagger(feature_func=GetFeatures(features_vector=features))
    model.train(train_es_prep[:500], 'crfTagger.mdl')  # Usando un subconjunto para rapidez
    prediction = model.tag_sents(dev_es_tokens[:100])  # Evaluamos en un subconjunto

    _, prediction_tags = sep_token_tag(prediction)

    feat_results = evaluate_tagger_performance(dev_es_tags[:100], prediction_tags, features)

    results_es = pd.concat([results_es, pd.DataFrame([feat_results])], ignore_index=True)

results_es

NameError: name 'train_es_prep' is not defined

A partir dels resultats podem observar que:

- La "balanced accuracy" és alta en tots els casos, indicant que el model ubica bastant bé on hi ha entitats
- El marge de millora es troba en detectar correctament les entitats i el seu tipus
- Les funcions que més milloren el model individualment són els prefixos i el Pos-Tag
- La longitud aporta millora en models amb més funcions actives
- Les Stop-Words no aporten pràcticament res

Per l'espanyol, seleccionem la combinació de funcions [1,1,1,0] (prefixos, pos-tag, longitud).

### Anàlisi de "feature functions" en el model de l'idioma holandés

In [ ]:
# Esta operación puede tardar bastante tiempo
# results_ned = features_grid_search(train_ned_prep, dev_ned_tokens, dev_ned_tags)

# Para ahorrar tiempo, probamos solo algunas configuraciones relevantes
test_features = [[0,0,0,0], [1,0,0,0], [0,1,0,0], [0,0,1,0], [0,0,0,1], [1,1,0,0], [1,1,1,0], [1,1,1,1]]
results_ned = pd.DataFrame(columns=['Features', 'Balanced accuracy', 'Total entities', 'Entities correct', 
                                  'LOC correct', 'MISC correct', 'ORG correct', 'PER correct', 'Entities invented'])

for features in test_features:
    model = nltk.tag.CRFTagger(feature_func=GetFeatures(features_vector=features))
    model.train(train_ned_prep[:500], 'crfTagger.mdl')  # Usando un subconjunto para rapidez
    prediction = model.tag_sents(dev_ned_tokens[:100])  # Evaluamos en un subconjunto

    _, prediction_tags = sep_token_tag(prediction)

    feat_results = evaluate_tagger_performance(dev_ned_tags[:100], prediction_tags, features)

    results_ned = pd.concat([results_ned, pd.DataFrame([feat_results])], ignore_index=True)

results_ned

Per l'holandès, també observem:

- El rang de millora és superior que en espanyol, tant en "accuracy" com en detecció d'entitats
- Les funcions 1 (prefixos) i 2 (pos-tag) tenen el major impacte individual
- La funció 3 (longitud) millora lleugerament, però la 4 (stop-words) no aporta
- MISC es prediu millor que en espanyol, mentre que ORG es prediu pitjor

Seleccionem també la combinació [1,1,1,0] per l'holandès.

### Prova d'afegir llistes externes de noms i localitzacions

Finalment, provem si afegir la cinquena funció extra (llistes externes) millora els resultats.

In [ ]:
def list_comparation(train, val_tokens, val_tags):
    '''
    Compare the performance of the tagger with and without lists
    '''
    results = pd.DataFrame(columns=['Lists', 'Balanced accuracy', 'Total entities', 'Entities correct', 
                                    'LOC correct', 'MISC correct', 'ORG correct', 'PER correct', 'Entities invented'])

    # Sin listas
    model = nltk.tag.CRFTagger(feature_func=GetFeatures(features_vector=[1, 1, 1, 0], lists=False))
    model.train(train[:500], 'crfTagger.mdl')  # Subsample para rapidez
    prediction = model.tag_sents(val_tokens[:100])

    _, prediction_tags = sep_token_tag(prediction)

    feat_results = evaluate_tagger_performance(val_tags[:100], prediction_tags, "No lists")

    results = pd.concat([results, pd.DataFrame([feat_results])], ignore_index=True)

    # Con listas
    model = nltk.tag.CRFTagger(feature_func=GetFeatures(features_vector=[1, 1, 1, 0], lists=True))
    model.train(train[:500], 'crfTagger.mdl')
    prediction = model.tag_sents(val_tokens[:100])

    _, prediction_tags = sep_token_tag(prediction)

    feat_results = evaluate_tagger_performance(val_tags[:100], prediction_tags, "Lists")

    results = pd.concat([results, pd.DataFrame([feat_results])], ignore_index=True)

    return results

# Comparación para español
results_lists_es = list_comparation(train_es_prep, dev_es_tokens, dev_es_tags)
print("Resultados para español con y sin listas:")
print(results_lists_es)

# Comparación para holandés
results_lists_ned = list_comparation(train_ned_prep, dev_ned_tokens, dev_ned_tags)
print("\nResultados para holandés con y sin listas:")
print(results_lists_ned)

Després d'analitzar els resultats amb i sense llistes, observem:

- Les llistes milloren significativament el percentatge d'entitats predites correctament
- En espanyol, millora especialment la detecció de localitzacions i persones
- En holandès, la millora es concentra en localitzacions
- Les llistes també redueixen el nombre d'entitats inventades

Per tant, utilitzarem les llistes en ambdós models finals.

## 2 - Experimentació de context

Un cop decidides les "feature functions", experimentem amb diferents contexts. Fins ara només hem donat informació del token a predir, ara provarem d'incloure informació de les paraules del voltant.

Provarem les següents combinacions de context:
- Paraula actual
- Paraula anterior, paraula actual
- Paraula dos cops anterior, paraula anterior, paraula actual
- Paraula actual, paraula següent
- Paraula actual, paraula següent, paraula dos cops següent
- Paraula anterior, paraula actual, paraula següent
- Paraula anterior, paraula actual, paraula següent, paraula dos cops següent
- Paraula dos cops anterior, paraula anterior, paraula actual, paraula següent
- Paraula dos cops anterior, paraula anterior, paraula actual, paraula següent, paraula dos cops següent

In [ ]:
def word_vector_search(train, val_tokens, val_tags):
    '''
    Perform a grid search over the word vector
    '''
    word_combinations = [
        [0, 0, 1, 0, 0],  # Paraula actual
        [0, 1, 1, 0, 0],  # Paraula anterior, paraula actual
        [1, 1, 1, 0, 0],  # Paraula dos cops anterior, paraula anterior, paraula actual
        [0, 0, 1, 1, 0],  # Paraula actual, paraula següent
        [0, 0, 1, 1, 1],  # Paraula actual, paraula següent, paraula dos cops següent
        [0, 1, 1, 1, 0],  # Paraula anterior, paraula actual, paraula següent
        [0, 1, 1, 1, 1],  # Paraula anterior, paraula actual, paraula següent, paraula dos cops següent
        [1, 1, 1, 1, 0],  # Paraula dos cops anterior, paraula anterior, paraula actual, paraula següent
        [1, 1, 1, 1, 1]   # Paraula dos cops anterior, paraula anterior, paraula actual, paraula següent, paraula dos cops següent
    ]

    results = pd.DataFrame(columns=['Word vector', 'Balanced accuracy', 'Total entities', 'Entities correct', 
                                    'LOC correct', 'MISC correct', 'ORG correct', 'PER correct', 'Entities invented'])

    # Para cada combinación de word vector
    for i, word_vector in enumerate(word_combinations):
        model = nltk.tag.CRFTagger(feature_func=GetFeatures(features_vector=[1, 1, 1, 0], lists=True, word_vector=word_vector))
        model.train(train[:500], 'crfTagger.mdl')
        prediction = model.tag_sents(val_tokens[:100])

        _, prediction_tags = sep_token_tag(prediction)

        feat_results = evaluate_tagger_performance(val_tags[:100], prediction_tags, f"Context {i+1}")

        results = pd.concat([results, pd.DataFrame([feat_results])], ignore_index=True)

    return results

In [ ]:
# Esta operación puede tardar bastante tiempo
# Probamos versiones simplificadas para español
context_combinations = [
    [0, 0, 1, 0, 0],  # Solo palabra actual
    [0, 1, 1, 0, 0],  # Palabra anterior + actual
    [0, 1, 1, 1, 0],  # Palabra anterior + actual + siguiente
    [1, 1, 1, 1, 1]   # Contexto completo
]

results_context_es = pd.DataFrame(columns=['Word vector', 'Balanced accuracy', 'Total entities', 'Entities correct', 
                                          'LOC correct', 'MISC correct', 'ORG correct', 'PER correct', 'Entities invented'])

for i, word_vector in enumerate(context_combinations):
    model = nltk.tag.CRFTagger(feature_func=GetFeatures(features_vector=[1, 1, 1, 0], lists=True, word_vector=word_vector))
    model.train(train_es_prep[:500], 'crfTagger.mdl')
    prediction = model.tag_sents(dev_es_tokens[:100])

    _, prediction_tags = sep_token_tag(prediction)

    feat_results = evaluate_tagger_performance(dev_es_tags[:100], prediction_tags, f"Context {i+1}")

    results_context_es = pd.concat([results_context_es, pd.DataFrame([feat_results])], ignore_index=True)

results_context_es

In [ ]:
# Probamos versiones simplificadas para holandés
results_context_ned = pd.DataFrame(columns=['Word vector', 'Balanced accuracy', 'Total entities', 'Entities correct', 
                                           'LOC correct', 'MISC correct', 'ORG correct', 'PER correct', 'Entities invented'])

for i, word_vector in enumerate(context_combinations):
    model = nltk.tag.CRFTagger(feature_func=GetFeatures(features_vector=[1, 1, 1, 0], lists=True, word_vector=word_vector))
    model.train(train_ned_prep[:500], 'crfTagger.mdl')
    prediction = model.tag_sents(dev_ned_tokens[:100])

    _, prediction_tags = sep_token_tag(prediction)

    feat_results = evaluate_tagger_performance(dev_ned_tags[:100], prediction_tags, f"Context {i+1}")

    results_context_ned = pd.concat([results_context_ned, pd.DataFrame([feat_results])], ignore_index=True)

results_context_ned

Dels resultats observem:

- Afegir funcions del context millora el rendiment del model
- Donar massa context pot empitjorar els resultats
- Per ambdós idiomes, la millor combinació és: **Paraula anterior, paraula actual, paraula següent** [0,1,1,1,0]
- Aquest context millora tots els tipus d'entitats i disminueix el nombre d'entitats inventades

Els nostres models finals tindran els següents paràmetres:
- features_vector = [1, 1, 1, 0] (prefixos, pos-tag, longitud)
- lists = True (utilitzem llistes externes)
- word_vector = [0, 1, 1, 1, 0] (paraula anterior, actual i següent)

## 3 - Experimentació de les codificacions

Ara provarem els models finals amb diferents codificacions per escollir-ne una per cada idioma. Provarem:
- IO
- BIO
- BIOE
- BIOS
- BIOES

In [ ]:
def train_to_IO(train):
    '''
    Convert the train to IO format
    '''
    train_io = []
    for sent in train:
        sent_io = []
        for word in sent:
            new_tag = re.sub(r'\bB-', 'I-', word[1])
            sent_io.append((word[0], new_tag))
        train_io.append(sent_io)
    return train_io

def train_to_BIOE(train):
    '''
    Convert the train to BIOE format
    '''
    train_bioe = []
    for sent in train:
        sent_bioe = []
        for i in range(len(sent)):
            tag = sent[i][1]
            if tag[0] == 'I' and (i == len(sent) - 1 or sent[i + 1][1][0] != 'I' or sent[i+1][1][2:] != tag[2:]):
                tag = re.sub(r'\bI-', 'E-', tag)
            sent_bioe.append((sent[i][0], tag))
        train_bioe.append(sent_bioe)
    return train_bioe

def train_to_BIOS(train):
    '''
    Convert the train to BIOS format
    '''
    train_bios = []
    for sent in train:
        sent_bios = []
        for i in range(len(sent)):
            tag = sent[i][1]
            if tag[0] == 'B' and (i == len(sent) - 1 or sent[i + 1][1][0] != 'I' or sent[i+1][1][2:] != tag[2:]):
                tag = re.sub(r'\bB-', 'S-', tag)
            sent_bios.append((sent[i][0], tag))
        train_bios.append(sent_bios)
    return train_bios

def train_to_BIOES(train):
    '''
    Convert the train to BIOES format
    '''
    train_bios = train_to_BIOS(train)
    train_bioes = train_to_BIOE(train_bios)
    return train_bioes

# Crear versiones con diferentes codificaciones (usando una muestra para rapidez)
esp_train_sample = train_es_prep[:500]
esp_train_IO = train_to_IO(esp_train_sample)
esp_train_BIOE = train_to_BIOE(esp_train_sample)
esp_train_BIOS = train_to_BIOS(esp_train_sample)
esp_train_BIOES = train_to_BIOES(esp_train_sample)

ned_train_sample = train_ned_prep[:500]
ned_train_IO = train_to_IO(ned_train_sample)
ned_train_BIOE = train_to_BIOE(ned_train_sample)
ned_train_BIOS = train_to_BIOS(ned_train_sample)
ned_train_BIOES = train_to_BIOES(ned_train_sample)

In [ ]:
def cod_grid_search(trains, val_tokens, val_tags, cod):
    '''
    Perform a grid search over the codification
    '''
    results = pd.DataFrame(columns=['Codification', 'Balanced accuracy', 'Total entities', 'Entities correct', 
                                    'LOC correct', 'MISC correct', 'ORG correct', 'PER correct', 'Entities invented'])

    for i in range(len(trains)):
        model = nltk.tag.CRFTagger(feature_func=GetFeatures(features_vector=[1, 1, 1, 0], lists=True, word_vector=[0, 1, 1, 1, 0]))
        model.train(trains[i], 'crfTagger.mdl')
        prediction = model.tag_sents(val_tokens[:100])

        _, prediction_tags = sep_token_tag(prediction)

        feat_results = evaluate_tagger_performance(val_tags[:100], prediction_tags, cod[i])

        results = pd.concat([results, pd.DataFrame([feat_results])], ignore_index=True)

    return results

# Probamos diferentes codificaciones
cod_train_esp = [esp_train_sample, esp_train_IO, esp_train_BIOE, esp_train_BIOS, esp_train_BIOES]
cod_train_ned = [ned_train_sample, ned_train_IO, ned_train_BIOE, ned_train_BIOS, ned_train_BIOES]
cod = ['BIO','IO', 'BIOE', 'BIOS', 'BIOES']

results_cod_es = cod_grid_search(cod_train_esp, dev_es_tokens, dev_es_tags, cod)
print("Resultados de codificaciones para español:")
print(results_cod_es)

results_cod_ned = cod_grid_search(cod_train_ned, dev_ned_tokens, dev_ned_tags, cod)
print("\nResultados de codificaciones para holandés:")
print(results_cod_ned)

Després de provar les diferents codificacions observem:

**Per a l'espanyol:**
- No hi ha pràcticament diferència entre les codificacions
- La codificació BIOES dona la millor "balanced accuracy" i precisió en entitats
- Però BIOES augmenta lleugerament el nombre d'entitats inventades
- Decidim mantenir la codificació original BIO per simplicitat

**Per a l'holandès:**
- Totes les codificacions empitjoren respecte l'original (BIO)
- Mantenim també la codificació BIO

La conclusió és que per ambdós idiomes utilitzarem la codificació BIO.

## 4 - Anàlisi dels models definitius

Ara que hem definit els models finals, els entrenarem amb tots els paràmetres seleccionats i els avaluarem en la partició de test:

- Features: prefixos, pos-tag i longitud [1,1,1,0]
- Llistes externes: sí
- Context: paraula anterior, actual i següent [0,1,1,1,0]
- Codificació: BIO

In [ ]:
# Entrenamos y evaluamos el modelo final para español
model_esp = nltk.tag.CRFTagger(feature_func=GetFeatures(features_vector=[1, 1, 1, 0], lists=True, word_vector=[0, 1, 1, 1, 0]))
model_esp.train(train_es_prep, 'esp_model.mdl')
prediction_esp = model_esp.tag_sents(test_es_tokens)

_, prediction_esp_tags = sep_token_tag(prediction_esp)

results_esp_final, errors_esp = evaluate_tagger_performance(test_es_tags, prediction_esp_tags, "Español Final", errors=True)
print("Resultados del modelo español en test:")
for key, value in results_esp_final.items():
    print(f"{key}: {value}")

In [ ]:
# Entrenamos y evaluamos el modelo final para holandés
model_ned = nltk.tag.CRFTagger(feature_func=GetFeatures(features_vector=[1, 1, 1, 0], lists=True, word_vector=[0, 1, 1, 1, 0]))
model_ned.train(train_ned_prep, 'ned_model.mdl')
prediction_ned = model_ned.tag_sents(test_ned_tokens)

_, prediction_ned_tags = sep_token_tag(prediction_ned)

results_ned_final, errors_ned = evaluate_tagger_performance(test_ned_tags, prediction_ned_tags, "Holandés Final", errors=True)
print("Resultados del modelo holandés en test:")
for key, value in results_ned_final.items():
    print(f"{key}: {value}")

### Anàlisi d'errors

Examinem alguns errors específics per entendre millor les limitacions dels models:

In [ ]:
import random
random.seed(21)
# Escogemos un error aleatorio del modelo español
if errors_esp and len(errors_esp) > 0:
    error = random.choice(errors_esp)
    
    idx = error[0]
    entity = error[1]
    
    sentence = [token[0] for token in test_es[idx]]
    
    print("Frase original:", sentence)
    print("Etiquetas reales:", [tag for tag in test_es_tags[idx]])
    print("Etiquetas predichas:", [tag[1] for tag in prediction_esp[idx]])
    print("\nError en la entidad:", entity)

In [ ]:
random.seed(23)
# Escogemos un error aleatorio del modelo holandés
if errors_ned and len(errors_ned) > 0:
    error = random.choice(errors_ned)
    
    idx = error[0]
    entity = error[1]
    
    sentence = [token[0] for token in test_ned[idx]]
    
    print("Frase original:", sentence)
    print("Etiquetas reales:", [tag for tag in test_ned_tags[idx]])
    print("Etiquetas predichas:", [tag[1] for tag in prediction_ned[idx]])
    print("\nError en la entidad:", entity)

### Conclusions

**Model Espanyol:**
- Excel·lent "accuracy" balancejada, indicant bona capacitat per ubicar entitats
- Aproximadament 4 de cada 5 entitats són detectades perfectament
- És especialment bo en detectar persones (90%), seguit de localitzacions i organitzacions
- El tipus MISC costa més de detectar (50% d'encert)
- Els errors més comuns són:
  - Paraules estranyes que es prediuen amb un tipus incorrecte
  - Entitats extenses que no s'acaben de predir correctament

**Model Holandès:**
- Resultats una mica inferiors al model espanyol, però igualment bons
- 3 de cada 4 entitats són identificades perfectament
- També excel·lent en detectar persones i localitzacions
- Més problemes amb organitzacions que el model espanyol
- Millor detecció del tipus MISC que el model espanyol
- Els errors més comuns continuen sent sobre el tipus d'entitat, no sobre la ubicació

En definitiva, hem creat dos models funcionals que detecten la gran majoria d'entitats i, en la majoria dels casos, identifiquen correctament el seu tipus.